In [31]:
SURVEY = 'hltds_wide'

if SURVEY == 'hltds_wide':
    area = 19.04
elif SURVEY == 'hltds_deep':
    area = 4.2
else:
    raise ValueError('Invalid survey')

In [32]:
import os
import sys

import numpy as np
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import colors
from hydra import initialize, compose
import pickle
from glob import glob
from pprint import pprint
from tqdm import tqdm
import datetime

# set paths to various directories based on the machine this code is being executed on
try:
    with initialize(version_base=None, config_path='config'):  
        config = compose(config_name=f'config_{SURVEY}.yaml')  # overrides=['machine=uzay']
except:
    with initialize(version_base=None, config_path='../../config'):  
        config = compose(config_name=f'config_{SURVEY}.yaml')  # overrides=['machine=uzay']

array_dir, data_dir, figure_dir, pickle_dir, repo_dir  = config.machine.array_dir, config.machine.data_dir, config.machine.figure_dir, config.machine.pickle_dir, config.machine.repo_dir

# enable use of local modules
if repo_dir not in sys.path:
    sys.path.append(repo_dir)

# set matplotlib style
plt.style.use(f'{repo_dir}/mejiro/mplstyle/science.mplstyle')

from mejiro.utils import util

survey_params = util.hydra_to_dict(config.survey)

In [33]:
pipeline_params = util.hydra_to_dict(config.pipeline)
debugging = pipeline_params['debugging']

if debugging:
    pipeline_dir = f'{config.machine.pipeline_dir}_{SURVEY}'
else:
    pipeline_dir = config.machine.pipeline_dir

# Detectable and candidate lens counts

Number of detectable strong lenses per square degree of simulated area, candidate strong lenses

In [34]:
# combine CSVs from each run into one CSV
data_dir = os.path.join(pipeline_dir, '00')

detectable_csv = os.path.join(data_dir, 'detectable_pop.csv')
util.delete_if_exists(detectable_csv)
detectable_df = util.combine_all_csvs(data_dir, 'detectable', detectable_csv)

if survey_params['total_population']:
    total_csv = os.path.join(data_dir, 'total_pop.csv')
    util.delete_if_exists(total_csv)
    total_df = util.combine_all_csvs(data_dir, 'total', total_csv)

Wrote combined CSV to /data/bwedig/mejiro/pipeline_hltds_wide/00/detectable_pop.csv
Wrote combined CSV to /data/bwedig/mejiro/pipeline_hltds_wide/00/total_pop.csv


In [35]:
survey_area = survey_params['area']
runs = survey_params['runs']

total_area = survey_area * runs
print(f'{runs} run(s) of {survey_area} sq. deg. each gives {total_area:.2f} sq. deg. total')

12 run(s) of 0.1 sq. deg. each gives 1.20 sq. deg. total


In [36]:
print(f'{len(detectable_df)} detectable strong lenses')

if survey_params['total_population']:
    print(f'{len(total_df)} total candidate strong lenses')
    fraction_detectable = len(detectable_df) / len(total_df)
    print(f'Fraction of candidate strong lenses that are detectable: {fraction_detectable:.5f}')

2055 detectable strong lenses
146715 total candidate strong lenses
Fraction of candidate strong lenses that are detectable: 0.01401


In [37]:
det_per_sq_deg = len(detectable_df) / total_area
print(f'Detectable strong lenses per sq. deg.: {det_per_sq_deg:.2f}')

if survey_params['total_population']:
    total_per_sq_deg = len(total_df) / total_area
    print(f'Candidate strong lenses per sq. deg.: {round(total_per_sq_deg)}')

Detectable strong lenses per sq. deg.: 1712.50
Candidate strong lenses per sq. deg.: 122262


In [38]:
det_per_exposure = det_per_sq_deg * 0.281
print(f'Detectable strong lenses per exposure: {det_per_exposure:.2f}')

Detectable strong lenses per exposure: 481.21


In [39]:
snr_threshold = 200
high_snr_df = detectable_df[detectable_df['snr'] > snr_threshold]
high_snr_det_per_sq_deg = len(high_snr_df) / total_area
high_snr_det_per_exposure = high_snr_det_per_sq_deg * 0.281

print(f'{len(high_snr_df)} characterizable strong lenses (SNR > {snr_threshold})')
print(f'{high_snr_det_per_sq_deg:.2f} characterizable per sq. deg.')
print(f'Characterizable strong lenses per exposure: {high_snr_det_per_exposure:.2f}')

99 characterizable strong lenses (SNR > 200)
82.50 characterizable per sq. deg.
Characterizable strong lenses per exposure: 23.18


In [40]:
print(f'{round(det_per_sq_deg * area)} detectable strong lenses in {SURVEY}')
print(f'{round(high_snr_det_per_sq_deg * area)} characterizable strong lenses in {SURVEY}')

32606 detectable strong lenses in hltds_wide
1571 characterizable strong lenses in hltds_wide
